# Practice Random Forest Pipeline/Workflow

In [10]:
# Imports
import pandas as pd
import numpy as np
import random
import string

### Create fake/dummy data

In [11]:
row_num = 5000

In [12]:
# Create columns names for dataset
columns = [''.join(random.choices(string.ascii_uppercase + string.digits, k=5)) for x in range(380)]

In [13]:
# Add Thyroid cancer to columns
columns.insert(0, 'userId')
# Add userId to columns
columns.append('thyroid_cancer')

In [34]:
# Create dataframe rows with different possible data types

dictionary = {}

for item in columns:

    column_options = [list(np.random.randint(0, 2, row_num)), np.random.randint(0, 11, row_num), np.random.randint(0, 4, row_num),
                      np.random.random_sample(size = row_num), np.random.uniform(low=0, high=3000, size=row_num), np.random.uniform(low=0, high=150, size=row_num)]

    dictionary[item] = random.choice(column_options)


In [36]:
# Create the pandas DataFrame
df = pd.DataFrame(dictionary)

In [16]:
# Change userId values
df['userId'] = [x for x in range(row_num)]

# Make userId index
df.set_index('userId', inplace=True)

In [17]:
# Add missing values
for col in df.columns:
    df.loc[df.sample(frac=0.1).index, col] = np.nan

In [18]:
# Make thyroid cancer data binary with no missing values
df['thyroid_cancer'] = [random.randint(0, 1) for x in range(row_num)]

### Data preprocessing

In [37]:
df


,userId,B9E7W,X9F8H,IWAN8,FODG8,PR84W,R1WI2,RGLSN,C0UHS,Q2RMM,...,AHA5P,3HZAJ,RXT0H,5HGWR,OKO67,NT9BD,JNXCE,C2PP7,HJBV8,thyroid_cancer
0,0,51.651866,0.264715,4,915.714065,3,1,6,137.428775,1,...,1,21.367925,3,1,19.177256,0,132.591508,2632.979803,121.061003,1
1,0,7.023647,0.516846,5,1456.495139,7,3,10,134.954853,3,...,3,117.189002,1,1,121.364933,1,26.053990,1241.527997,9.390588,0
2,1,15.838792,0.264070,5,372.008027,9,2,1,76.980645,1,...,2,72.505988,2,3,27.436234,0,118.747144,2320.805935,72.537729,0
3,0,71.239033,0.868426,7,405.704651,0,0,8,122.134039,3,...,1,19.951426,2,3,106.728728,1,41.453929,2033.348592,49.064759,1
4,1,135.631845,0.099530,1,459.078405,1,2,1,68.036007,2,...,1,3.930842,2,3,104.386260,1,114.267541,1443.451922,33.864787,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1,103.391164,0.054728,8,1549.333396,8,2,4,57.823025,0,...,2,133.075872,3,2,143.434557,0,71.445206,592.162594,5.230247,1
4996,1,72.703128,0.199328,0,166.798940,4,1,9,115.848432,1,...,0,50.989074,0,1,136.791721,0,57.440018,1242.776817,90.797848,1
4997,1,120.605469,0.963607,8,161.600108,3,0,9,131.278664,2,...,2,121.568081,3,0,148.934440,1,113.646151,997.605150,135.476835,0
4998,1,17.548733,0.803683,3,2281.155098,2,1,2,4.889897,2,...,2,77.657040,1,2,117.855228,1,149.226111,1144.391626,104.369759,1


In [20]:
# Make sure thyroid cancer column has been added

In [21]:
# Make sure diseases descriptions are the column names rather than just phecodes

In [22]:
# Manual feature selection - use mutual information and consider data leakage.

### Pipeline

In [ ]:
import

In [ ]:
# Create test-train split


Feature selection: MRMR or Recursive Feature Elimination (RFE) or Boruta

You could consider feature selection a hyperparameter. Consequently, you can include it as part of a cross validation grid search to obtain optimum hyperparameters for the model in question.

https://medium.com/data-science-reporter/feature-selection-via-grid-search-in-supervised-models-4dc0c43d7ab1

Preproccessing:

- Clean data
- Add thyroid cancer
- Colum names
- Manual feature selection (data leakage - look at mutual information)


Pipeline:

Test-Train split

1. Imputation
2. Feature scaling (not always)
3. Oversampling (not always)
4. Model selection
5. Cross validation grid search for hyperparameters and feature selection
6. Build Model
7.